In [2]:
import pandas as pd
import numpy as np
import os

import sys
from tqdm import tqdm

import nltk

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
import os

from sklearn.metrics import recall_score, precision_score

sys.path.insert(0,'/home/roshansk/YelpAnalysis/Datasets/')



from createDataset import *

%load_ext autoreload
%autoreload 2

import xlsxwriter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
lexiconA = pd.read_csv('/home/roshansk/YelpAnalysis/Lexicons/FFN/NN_Yelp_subset_Lexicon.csv')

lexiconB = pd.read_csv('/home/roshansk/YelpAnalysis/Lexicons/LSTM/nyelp_subset_lstm_div.csv')

lexiconA = lexiconA.dropna()
lexiconB = lexiconB.dropna()

In [19]:
lexiconA.head()

,word,scoreA,wordCount,rankA
0,daith,31.903006,5,0
1,misti,31.836288,3,1
2,cutest,31.457407,2,2
3,adorable,31.226568,18,3
4,bookmark,30.251474,1,4


In [6]:
lexiconA['rankA'] = list(range(len(lexiconA)))
lexiconB['rankB'] = list(range(len(lexiconB)))

if 'corr' in lexiconA.columns:
    lexiconA.rename({'corr':'score'},axis = 1,inplace = True)
    
if 'corr' in lexiconB.columns:
    lexiconB.rename({'corr':'score'},axis = 1, inplace = True)

lexiconA = lexiconA.rename({'score':'scoreA'},axis = 1)
lexiconB = lexiconB.rename({'score':'scoreB'},axis = 1)

if 'wordCount' in lexiconB.columns:
    lexiconB.drop('wordCount',axis = 1, inplace = True)

lexicon = lexiconA.merge(lexiconB, on='word')

lexicon['rankDiff']  = lexicon.rankA - lexicon.rankB
lexicon.rankDiff = np.abs(lexicon.rankDiff)

lexicon.drop('scoreA',inplace = True, axis = 1)
lexicon.drop('scoreB',inplace = True, axis = 1)

print(len(lexicon), len(lexiconA), len(lexiconB))

24849 25023 24849


In [4]:
### Only once in training set
# lexicon = lexicon[lexicon.wordCount==1]
# print(len(lexicon))

In [7]:
lexicon.head()

,word,wordCount,rankA,Unnamed: 0,occurrence,rankB,rankDiff
0,daith,5,0,12303,5.0,22381,22381
1,misti,3,1,20252,3.0,13157,13156
2,cutest,2,2,7643,2.0,18656,18654
3,adorable,18,3,18290,18.0,5521,5518
4,bookmark,1,4,12569,1.0,22644,22640


### Method A

In [8]:
lexicon.sort_values('rankA',ascending=True,inplace=True)
lexicon.head()

,word,wordCount,rankA,Unnamed: 0,occurrence,rankB,rankDiff
0,daith,5,0,12303,5.0,22381,22381
1,misti,3,1,20252,3.0,13157,13156
2,cutest,2,2,7643,2.0,18656,18654
3,adorable,18,3,18290,18.0,5521,5518
4,bookmark,1,4,12569,1.0,22644,22640


In [9]:
topWordsA = lexicon.iloc[0:200]
bottomWordsA = lexicon.iloc[-200:]

In [10]:
lexicon.sort_values('rankB',ascending=True,inplace=True)
lexicon.head()

,word,wordCount,rankA,Unnamed: 0,occurrence,rankB,rankDiff
747,great,4157,747,0,4157.0,0,747
9350,good,5751,9357,1,5751.0,1,9356
180,love,1474,180,2,1474.0,2,178
137,amazing,1152,137,3,1152.0,3,134
377,delicious,1174,377,4,1174.0,4,373


In [11]:
lexicon.rankDiff.describe()

count    24849.000000
mean      7954.022093
std       5670.726531
min          0.000000
25%       3189.000000
50%       6900.000000
75%      12120.000000
max      24858.000000
Name: rankDiff, dtype: float64

In [12]:
topWordsB = lexicon.iloc[0:200]
bottomWordsB = lexicon.iloc[-200:]

In [13]:
topWordsA.sort_values('rankDiff',ascending=False).head(20)

,word,wordCount,rankA,Unnamed: 0,occurrence,rankB,rankDiff
50,superb,72,50,411,72.0,24785,24735
174,tucked,29,174,18519,29.0,23305,23131
36,bitesize,1,36,12433,1.0,22757,22721
156,terriers,1,156,11945,1.0,22873,22717
187,cremini,1,187,11979,1.0,22893,22706
4,bookmark,1,4,12569,1.0,22644,22640
84,lawrenceville,4,84,12527,4.0,22664,22580
111,yummies,1,111,12522,1.0,22659,22548
0,daith,5,0,12303,5.0,22381,22381
76,meilleurs,1,76,10993,1.0,22018,21942


In [14]:
topWordsB.sort_values('rankDiff',ascending=False).head(20)

,word,wordCount,rankA,Unnamed: 0,occurrence,rankB,rankDiff
24839,flavorless,53,25013,168,53.0,155,24858
24791,garbage,65,24965,158,65.0,114,24851
24410,empty,339,24584,131,339.0,165,24419
24238,frozen,121,24411,162,121.0,157,24254
23968,unhealthy,4,24140,208,4.0,154,23986
23961,false,50,24133,251,50.0,194,23939
23320,comped,44,23491,247,44.0,191,23300
23235,unfortunate,40,23406,230,40.0,176,23230
23028,inconsistent,19,23199,182,19.0,135,23064
22862,sadly,54,23033,206,54.0,149,22884


#### Write to File

In [15]:
outputFile = 'Test2'
modelAName = 'A : FNN yelp_subset'
modelBName = 'B : LSTM yelp_subet'

In [16]:
writer = pd.ExcelWriter(f'{outputFile}.xlsx', engine='xlsxwriter')

In [17]:
topWordsA.sort_values('rankDiff',ascending=False).head(20).to_excel(writer, sheet_name='Sheet1', startrow=4)

topWordsA.sort_values('rankDiff',ascending=False).tail(20).to_excel(writer, sheet_name='Sheet1', startrow=4, startcol = 10)

topWordsB.sort_values('rankDiff',ascending=False).head(20).to_excel(writer, sheet_name='Sheet1', startrow=28)

topWordsB.sort_values('rankDiff',ascending=False).tail(20).to_excel(writer, sheet_name='Sheet1', startrow=28, startcol = 10)

In [18]:

worksheet = writer.sheets['Sheet1']

worksheet.write(0, 0, modelAName)
worksheet.write(1, 0, modelBName)

worksheet.write(3, 0, 'Method A : Top 20 words')
worksheet.write(3, 10, 'Method A : Bottom 20 words')

worksheet.write(27, 0, 'Method B : Top 20 words')
worksheet.write(27, 10, 'Method B : Bottom 20 words')

writer.close()

#### Word Count Testing

In [40]:
sys.path.insert(0,'/home/roshansk/YelpAnalysis/Datasets/')
from createDataset import *

dataFolder = '/home/roshansk/YelpAnalysis/Datasets/'

import spacy
import itertools
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl')

def getWordCount(data, textVariable = 'text'):

    vectorizer = CountVectorizer(stop_words=[], tokenizer= nltk.word_tokenize)
    vectorizer.fit(data[textVariable])

    cvFit = vectorizer.transform(data[textVariable])
    wordList = list(vectorizer.vocabulary_.keys())

    counts = np.asarray(cvFit.sum(axis=0))[0]

    wordCount = []
    for i in range(len(wordList)):
        wordCount.append(counts[vectorizer.vocabulary_[wordList[i]]])
        
    df = pd.DataFrame({'word':wordList,'wordCount':wordCount})
    
    return df

In [10]:
trainDf, devDf, testDf = getData(dataFolder, 'yelp_subset')

In [37]:
def getSpacyWordCount(df):

    tokenList = []

    for i in tqdm(range(len(df))):

        doc = nlp(df.iloc[i]['text'].lower())
        words = [token.text for token in doc]

        tokenList.append(words)
        
    wordCount = list(itertools.chain.from_iterable(tokenList))
        
    word_freq = Counter(wordCount)
    
    df = pd.Series(word_freq).reset_index()
    df.columns = ['word','wordCount']
    
    return df
        

In [38]:
spacyWordCount = getSpacyWordCount(trainDf)

100%|██████████| 12452/12452 [00:09<00:00, 1265.73it/s]


In [41]:
wordCount = getWordCount(trainDf)

/home/roshansk/Covid_Bert/env/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [54]:
spacyWordList = set(spacyWordCount.word)
wordList = set(wordCount.word)

In [55]:
wordList.difference(spacyWordList)

{'just-as-good',
 "'sushi",
 'entrances/exits',
 'u.s.',
 're-fired',
 'eat/drink',
 'hands-down',
 'up-and-coming',
 'beer-and-vibe-and-chips',
 '8:30.',
 'old-time',
 'didnt',
 'yonge/bloor',
 "'fast",
 'like-minded',
 'too-symmetrical',
 '1100-1600',
 '1998.',
 'add-ons',
 'three-seat',
 'shoe-string',
 'apple-cheese-cranberry-panini',
 'spinach-artichoke',
 'co-pays',
 'would-because',
 'track-workers',
 'mid-afternoon',
 'ave.',
 "'nanny",
 'purée-like',
 'bacon/onion/egg',
 '-o',
 'post-infectious',
 'deep-fried',
 "'indian",
 'pre-cooked',
 'luke-warm',
 'salad/sandwich',
 'sat/sun',
 'half-bad',
 'it-love',
 '20/person',
 'first-rate',
 'werent',
 'community-land',
 'less-than-stellar',
 'van-filled',
 'one-off',
 '109/nt',
 "'so",
 'chic-ier',
 'ambiance/service',
 'mineo',
 'expedia/hotels.com',
 'mid-westerners',
 'delay-but',
 'fast-disappearing',
 'vegetarian/vegan',
 'potato/almond',
 'clean-white',
 '16oz',
 'drive-through',
 'ingwer-soja',
 'switch-ups',
 'bench/couch',

In [56]:
'ave.' in wordList

True

In [57]:
'ave.' in spacyWordList

False

In [58]:
os.getcwd()

'/home/roshansk/YelpAnalysis/LexiconEvaluation'